In [1]:
# partners: Victoria Peryniak and Jared Novoa
# pytorch math functions reference: https://pytorch.org/docs/stable/torch.html

In [2]:
import torch
import pandas as pd
import numpy

In [3]:
#PART 1
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

In [4]:
# create trainX and trainY, testX and testY
# the last column in the dataframes are the Y, every other column are the X
trainX = torch.from_numpy(train_df.iloc[:, 0:-1].values).to(torch.float64)
trainY = torch.from_numpy(train_df.iloc[:, [-1]].values).to(torch.float64)
testX = torch.from_numpy(test_df.iloc[:, 0:-1].values).to(torch.float64)
testY = torch.from_numpy(test_df.iloc[:, [-1]].values).to(torch.float64)

In [5]:
#Q1: about how many bytes does trainX consume?
trainX.nelement()*trainX.element_size()

83520

In [6]:
#Q2: what is the biggest difference we would have any one cell if we used float16 instead of float64?
trainX_16 = trainX.to(torch.float16)
trainX_64 = trainX_16.to(torch.float64)
# https://www.tutorialspoint.com/how-to-perform-element-wise-subtraction-on-tensors-in-pytorch
float(torch.max(torch.sub(trainX_64,trainX))) # param1 - param2

0.0

In [7]:
#Q3: is a CUDA GPU available on your VM?
torch.cuda.is_available()

False

In [8]:
#PART 2
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)

In [9]:
#Q4: what is the predicted number of deaths for the first census tract?
(testX[0] @ coef).item()

9.844

In [10]:
#Q5: what is the average number of predicted deaths, over the whole testX dataset?
torch.mean(testX @ coef).item()

12.073632183908048

In [11]:
#Q6: first, what is y when x is a tensor containing 0.0?
def f(x):
    return (x**2) - (8*x) + 19

x = torch.tensor(0.0, requires_grad=True)
y = f(x)
float(y)

19.0

In [12]:
#Q7: what x value minimizes y?
optimizer = torch.optim.SGD([x], lr=0.1)

for epoch in range(1000):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
x.item() 

3.999999523162842

In [13]:
#Part 4
coef = torch.zeros(10,1, dtype=torch.float64, requires_grad=True)

In [14]:
#Q8: what is the MSE (mean-square error) when we make predictions using this vector of zero coefficients?
#MSE formula found from: https://www.geeksforgeeks.org/python-mean-squared-error/
torch.mean(torch.square(torch.sub((trainX @ coef), trainY))).item()

197.8007662835249

In [15]:
#Part 4, Optimization
torch.manual_seed(544)

optimizer = torch.optim.SGD([coef], lr = 0.000002)
loss_func = torch.nn.MSELoss()

ds = torch.utils.data.TensorDataset(trainX, trainY) # dataset made up of trainx and trainy
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True) # splits dataset into batches and shuffles the data 

#I used ChatGPT for help writing this loop
for epoch in range(500):
    for x,y in dl:
        outputs = x @ coef #multiply trainX by coefs to get the predicted y's (output = our predicted y's)
        loss = loss_func(outputs,y) #measures loss between our predicted y's with the real y's
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [16]:
#Q9: what is the MSE over the training data, using the coefficients resulting from the above training?
torch.mean(torch.square(torch.sub((trainX @ coef), trainY))).item()

26.8113940147193

In [17]:
#Q10: what is the MSE over the test data?
torch.mean(torch.square(torch.sub((testX @ coef), testY))).item()

29.05854692548551